In [24]:
#imports
import tweepy 
from tweepy import API
from tweepy import Stream
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import json
import pymongo 
from pymongo import MongoClient
import twitter
from pprint import pprint
#import MongoClient
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime
from datasketch import MinHash, MinHashLSH, MinHashLSHForest


In [25]:
import time

# # # consumer key, secre, access token, secret
consumer_key = 'IhUBGNHl7MiOBUEYfOK5aowxd'
consumer_secret = '0Hptp62enFQm3BJmmcdlD6UqU2I5qcllW4ZUGicuo7kfjyrW3w'
access_token = '1047073360474247168-bdgc58jJh9qGq9l9iBXag1KWYrFCge'
access_secret = 'f2lqVPyXtPG4AF3Cp9EusAThFIBaE9Cc1iMeu57vNRGKf'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


In [ ]:
# # # connecting mongo

client = MongoClient()
db = client.tweet_db
twit_col = db.twit_col
geo_search = db.geo_search
geo_stream = db.geo_stream
api_stream = db.tweet_stream
## LSH 
lsh_m1 = db.lsh_m1
lsh_m2 = db.lsh_m2
## LSH Cursors
m1_cursor = lsh_m1.find()
m2_cursor = lsh_m2.find()
geoCount = 0
streamingCount = 0

## each row represent one of the 6 ten minute sessions
# first column = # of retweets
# second column = # of quotes
# third column = # of redundant tweets
# fourth column = # geo tagged data from Glasgow
# fifth column =  # total geo tagged data
# sixth column =  total tweets
total_count = np.zeros((6,6))
topic_count = np.zeros((6,6))
search_count = np.zeros((6,6))
geo_count = np.zeros((6,6))

# # # Twitter Credentials
class TwitterAuthenticator():
    
    def authenticateTwitter(self):
        
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)

        return auth
# # # Twitter Search API
class TwitterSearch():
    
    def __init__(self):
        self.auth = TwitterAuthenticator().authenticateTwitter()
        self.twitterClient = API(self.auth)
        
        #self.twitterUser = twitterUser
    
    def geoSearch(self, geoCode, maxTweets):
        tweets = []
        redundant_tweets = 0

        i=0
        start_time = datetime.datetime.now()
        end_time = start_time + datetime.timedelta(0,120)
            
        while end_time > datetime.datetime.now():

            for tweet in Cursor(api.search, q="", geocode = "%f,%f,%dkm" % (geoCode[0], geoCode[1], geoCode[2])).items(maxTweets):

                #column 0 = retweets, 1 = quotes, 2 = redundant, 3 = glasgow, 4 = geo total, 5 = all tweets

                #incrementing retweets counts
                total_count[i,0] += tweet.retweet_count
                search_count[i,0] += tweet.retweet_count


                #incrementing glasgow count
                total_count[i,3] += 1
                search_count[i,3] += 1

                #incrementing geo
                total_count[i,4] += 1
                search_count[i,4] += 1

                #incrementing total
                total_count[i,5] += 1
                search_count[i,5] += 1
                

                dictionary = {

                    'UserName': tweet.user.name,

                    'ScreenName': tweet.user.screen_name,
                    'id': tweet.id,
                    'id_str': tweet.id_str,

                    'TweetCreatedAt': tweet.created_at,
                    'TweetText': tweet.text,

                    'UserLocation': tweet.user.location,
                    'TweetCoordinates': tweet.coordinates,
                    'Geo': tweet.geo,
                    'Place': None,
                    'GeoEnabled': tweet.user.geo_enabled,

                    'QuoteCount': 0,
                    'RetweetCount': tweet.retweet_count,
                    'Retweeted': tweet.retweeted,
                    'FavoriteCount': tweet.favorite_count,
                    'Favorited': tweet.favorited,
                    'LanguageCode': tweet.lang,
                    'Source': tweet.source,

                    'Replied': tweet.in_reply_to_status_id_str
                    }
                try:

                    geo_search.insert(dictionary)
                except:
                    print("mongo ERROR")
                    pass

                ### query mongo db to check for redundancy

                tweet_cursor = api_stream.find()

                for document in tweet_cursor:
                    try:
                        #print ('--Redundancy check--')

                        if document["id_str"] == id_str:

                            # counting redundancy
                            total_count[i,2] += 1
                            search_count[i,2] += 1

                            ## if redundant then discount the count for glasgow
                            if document["UserLocation"] != None:
                                if "glasgow" in document["UserLocation"].lower():
                                    total_count[i,3] -= 1
                                    search_count[i,3] -= 1

                    except:


                        pass

                tweets.append(dictionary)
        
        np.savetxt('search_count.csv',geo_count,delimiter=',', fmt="%s")

           
                
    
#################################################################
### Twitter Streamer
class TopicStreamer():
    
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
        
    def streamTweets(self, hash_tags, geo_box):
        #authentication
        listener = TopicListener()
        auth = self.twitter_authenticator.authenticateTwitter()
        stream = Stream(auth, listener)
        
        stream.filter(track = hash_tags)

#################################################################
### Twitter Streamer by location
class LocationStreamer():
    
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
        
    def streamTweets(self, hash_tags, geo_box):
        #authentication
        listener = GeoListener()
        auth = self.twitter_authenticator.authenticateTwitter()
        stream = Stream(auth, listener)
        
        stream.filter(locations = geo_box)
        
        
#################################################################
# # # Twitter Stream Listener 
class TopicListener(StreamListener):
    
    def __init__(self):
        pass
    
    def on_data(self, data):
        
      
        for i in range(5):

            start_time = datetime.datetime.now()
            end_time = start_time + datetime.timedelta(0,600)
            
            while end_time > datetime.datetime.now():
                i+=1
                tweet_json = json.loads(data)  

                user_name = tweet_json['user']['name']
                screen_name = tweet_json['user']['screen_name']
                user_id = tweet_json['user']['id']
                user_location = tweet_json['user']['location']
                geo_enabled = tweet_json['user']['geo_enabled']
                if tweet.geo_enabled != None:
                    place = (tweet_json['place']['coordinates'])[0]
                else:
                    place = None
                    
                    
                
                #incrementing retweets counts
                total_count[i,0] += tweet_json['retweet_count']
                topic_count[i,0] += tweet_json['retweet_count']

                #incrementing quotes count
                total_count[i,1] += tweet_json['quote_count']
                topic_count[i,1] += tweet_json['quote_count']

                #incrementing glasgow count
                if user_location != None:
                    if "Glasgow" in user_location:
                        total_count[i,3] += 1
                        topic_count[i,3] += 1

                #incrementing geo
                if geo_enabled == True:
                    total_count[i,4] += 1
                    topic_count[i,4] += 1


                #incrementing total
                total_count[i,5] += 1 
                topic_count[i,4] += 1


                dictionary = {

                        'UserName': user_name,
                        'ScreenName': screen_name,
                        'id': user_id,
                        'id_str': tweet_json['id_str'],

                        'TweetCreatedAt': tweet_json['created_at'],
                        'TweetText': tweet_json['text'],

                        'UserLocation': user_location,
                        'TweetCoordinates': tweet_json['coordinates'],
                        'Geo': tweet_json['geo'],
                        'Place': place,
                        'GeoEnabled': geo_enabled,
                        
                        'QuoteCount': tweet_json['quote_count'],
                        'RetweetCount': tweet_json['retweet_count'],
                        'Retweeted': tweet_json['retweeted'],
                        'FavoriteCount': tweet_json['favorite_count'],
                        'Favorited': tweet_json['favorited'],
                        'LanguageCode': tweet_json['lang'],
                        'Source': tweet_json['source'],

                        'Replied': tweet_json['in_reply_to_status_id_str']
                       }
                try:
                        api_stream.insert(dictionary)
                except:
                        print("mongo ERROR")
                        pass

                np.savetxt('topic_count.csv',topic_count,delimiter=',', fmt="%s")
                if time.time() > end_time:
                    break
    
# # # Twitter Stream Listener 
class GeoListener(StreamListener):
    
    def __init__(self):
        pass
    
    def on_data(self, data):
               
        for i in range(5):
            start_time = datetime.datetime.now()
            end_time = start_time + datetime.timedelta(0,600)

            while end_time > datetime.datetime.now():
                tweet_json = json.loads(data)  

                user_name = tweet_json['user']['name']
                screen_name = tweet_json['user']['screen_name']
                user_id = tweet_json['user']['id']
                user_location = tweet_json['user']['location']
                geo_enabled = tweet_json['user']['geo_enabled']
                if tweet.geo_enabled != None:
                    place = (tweet_json['place']['coordinates'])[0]
                else:
                    place = None

                #incrementing retweets counts
                total_count[i,0] += tweet_json['retweet_count']
                geo_count[i,0] += tweet_json['retweet_count']
                #incrementing quotes count
                total_count[i,1] += tweet_json['quote_count']
                geo_count[i,1] += tweet_json['quote_count']
                #incrementing glasgow count
                if "glasgow" in user_location.lower():
                    total_count[i,3] += 1
                    geo_count[i,1] += tweet_json['quote_count']
                #incrementing geo
                if geo_enabled == True:
                    total_count[i,4] += 1
                    geo_count[i,1] += tweet_json['quote_count']
                #incrementing total
                total_count[i,5] += 1 
                geo_count[i,1] += tweet_json['quote_count']

                #adding to dictionary before sending it
                dictionary = {

                        'UserName': user_name,
                        'ScreenName': screen_name,
                        'id': user_id,
                        'id_str': tweet_json['id_str'],

                        'TweetCreatedAt': tweet_json['created_at'],
                        'TweetText': tweet_json['text'],

                        'UserLocation': user_location,
                        'TweetCoordinates': tweet_json['coordinates'],
                        'Geo': tweet_json['geo'],
                        'Place': place,
                        'GeoEnabled': geo_enabled,

                        'QuoteCount': tweet_json['quote_count'],
                        'RetweetCount': tweet_json['retweet_count'],
                        'Retweeted': tweet_json['retweeted'],
                        'FavoriteCount': tweet_json['favorite_count'],
                        'Favorited': tweet_json['favorited'],
                        'LanguageCode': tweet_json['lang'],
                        'Source': tweet_json['source'],

                        'Replied': tweet_json['in_reply_to_status_id_str']
                       }
                try:
                        geo_stream.insert(dictionary)
                except:
                        print("mongo ERROR")
                        pass

                tweet_cursor = twit_col.find()
                user_cursor = twit_col.distinct("id")

                for document in tweet_cursor:
                    try:
                        print ('--Redundancy check--')

                        if document["id_str"] == id_str:
                            print ('id_str:', document["id_str"])
                            print ('found a match')

                            # counting redundancy
                            total_count[i,2] += 1
                            geo_count[i,2] += 1

                            ## if redundant then discount the count for glasgow
                            if "glasgow" in document["UserLocation"].lower():
                                total_count[i,3] -= 1
                                geo_count[i,3] -= 1

                    except:
                        error = "ERROR"

                        pass


    def on_error(self, status):
        if status == 420:
            return False
        print(status)
        
# # # Saving the counts of retweets, quotes, redundant tweets, glasgow tweets, 
# # # geo true tweets and total tweets
class SaveCounts():
    
    def __init__(self):
        
        np.savetxt('total_count.csv',total_count,delimiter=',', fmt="%s")
        np.savetxt('topic_count.csv',topic_count,delimiter=',', fmt="%s")
        np.savetxt('geo_count.csv',geo_count,delimiter=',', fmt="%s")
        np.savetxt('search_count.csv',search_count,delimiter=',', fmt="%s")
    
# # # MAIN calls streaming functions and grouping
if __name__ == '__main__':

    geo_code = [55.8642, -4.2518, 10] # for search api
    geo_box = [-10.6147114784,50.0038872941,1.8031525989,50.0038872941] # bounding box for location filter 
    maxTweets = 2500
    hash_tags = ["glasgow"] # topics for streaming

    # # # twitter topic stream api
   
    twitter_streamer = TopicStreamer()
    twitter_streamer.streamTweets(hash_tags, geo_box)
    
    # # # twitter location streamer
    twitter_streamer = LocationStreamer()
    twitter_streamer.streamTweets(hash_tags, geo_box)                                     
                  
    # # # twitter search api
    twitterClient = TwitterSearch()
    twitterClient.geoSearch(geo_code, maxTweets)
    
    # # # Process counts and build charts 
    saveCounts = SaveCounts()
    saveCounts
    


    
    


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import re
# The following is going to take a 6x6 matrix CSV file and perform data analysis
# The CSV was created during the streaming and rest calls
#
# each row represent one of the 6 ten minute sessions

# # # Read in CSV containing all of the count data and build histograms for all
# each row represent one of the 6 ten minute sessions
class DataAnalysis():

    def __init__(self):
        
        self.data = np.loadtxt("C:/Users/Abhi/Downloads/total_count1.csv", delimiter=',') # comma separated
        
        self.time_period = np.arange(1,7)
        
        self.retweets = self.data[:,0]
        self.quotes = self.data[:,1]
        self.redundant = self.data[:,2]
        self.glasgow = self.data[:,3]
        self.geo_total = self.data[:,4]
        self.total = self.data[:,5]
        
    # first column = # of retweets  
    def quotesRetweets(self):
        fig = plt.figure()
        axes = plt.subplot(1,1,1)
        barWidth=0.25
        
        r1 = np.arange(len(self.time_period))
        r2 = [x + barWidth for x in r1]
        r3 = [x + barWidth for x in r2]
        
        plt.bar(r1, self.total, color='red', width=barWidth, edgecolor='white', label='Total tweets')
        plt.bar(r2, self.retweets, color='yellow', width=barWidth, edgecolor='white', label='Retweets')
        plt.bar(r3, self.quotes, color='green', width=barWidth, edgecolor='white', label='Quotes')

        plt.xticks([r + barWidth for r in range(len(self.time_period))], ['1', '2', '3', '4', '5','6'])
        plt.legend()
        axes.set_title('Total tweets compared to number of Retweets and Quotes')
        axes.set_ylabel('Number of Tweets')
        axes.set_xlabel('Time Period (ten mins)')
        plt.show()
        
    # third column = # of redundant tweets
    def redundantCount(self):
        fig = plt.figure()
        axes = plt.subplot(1,1,1)
        barWidth=0.50
        
        r1 = np.arange(len(self.time_period))
        r2 = [x + barWidth for x in r1]
        
        plt.bar(r1, self.total, color='red', width=barWidth, edgecolor='white', label='Total tweets')
        plt.bar(r2, self.redundant, color='green', width=barWidth, edgecolor='white', label='Redundant tweets')
        
        plt.xlabel('Total tweets vs redundant tweets', fontweight='bold')
        plt.xticks([r + barWidth for r in range(len(self.time_period))], ['1', '2', '3', '4', '5','6'])
        plt.legend()
        axes.set_title('Total tweets vs Redundant tweets')
        axes.set_ylabel('Number of Tweets')
        axes.set_xlabel('Time Period (ten mins)')
        plt.show()
    
    # fourth column = # geo tagged data from Glasgow
    def glasgowCount(self):
        fig = plt.figure()
        axes = plt.subplot(1,1,1)
        plt.bar(self.time_period, self.glasgow)
        
        axes.set_title('Total tweets from Glasgow per time period')
        axes.set_ylabel('Number of Tweets')
        axes.set_xlabel('Time Period (ten mins)')
        plt.show()
    # sixth column =  total tweets
    def totalCount(self):
        fig = plt.figure()
        axes = plt.subplot(1,1,1)
        plt.bar(self.time_period, self.total)
        
        axes.set_title('Total number of tweets per time period')
        axes.set_ylabel('Number of Tweets')
        axes.set_xlabel('Time Period (ten mins)')
        plt.show()
    


# # # MAIN
if __name__ == '__main__':

    #calling data analysis class to plot data
    analyze = DataAnalysis()
    analyze.quotesRetweets()
    analyze.redundantCount()
    analyze.glasgowCount()
    analyze.totalCount()
    
    


In [ ]:
import json
import pymongo 
from pymongo import MongoClient
import numpy as np
import matplotlib.pyplot as plt
import re
import pandas as pd
import time
from datasketch import MinHash, MinHashLSH, MinHashLSHForest


#count = 0
##query mongodb existing collection for say 500 tweets, create two new collections with those 500 tweets


client = MongoClient()
db = client.tweet_db
tweet_collection = db.tweet_collection
# test_lsh = db.test_lsh
lsh_m1 = db.lsh_m1
lsh_m2 = db.lsh_m2
# lsh_test = db.lsh_test

# tweet_cursor = test_lsh.find()
m1_cursor = lsh_m1.find()
m2_cursor = lsh_m2.find()
# print(tweet_cursor.count())

#Number of Permutations
permutations = 128

#Number of Recommendations to return
#num_recommendations = 1

# first index with id string and text
# then tokenize the text
# class IndexMongo(): 

# then send the table as m1 to lsh (lsh.insert)
            

# then identical or duplicated dataset

# use a loop to go through each tweet 
# query that tweet with lsh

# for all the matches, send them to a new collection
# for all the matches, delete them from LSH and dataset
# def insertTest:

def tokenize(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

def get_lsh(perms):
    
    counter = 0
    groups = []
    group_results = []
    minhash = []
    lsh = MinHashLSH(threshold=0.5,num_perm = 128)
    m1 = MinHash(num_perm=perms)
    m2 = MinHash(num_perm=perms)
    
    for x in m1_cursor:
        id1 = x['id_str']
        for word in x['text']:
            m1.update(word.encode('utf8'))
        lsh.insert(id1,m1)
        
    for y in m2_cursor:
        id2 = y['id_str']
        if lsh.__contains__(id2):
            for word in y['text']:
                m2.update(word.encode('utf8'))
            result = lsh.query(m2)
            lsh.remove(id2)
            groups.append(counter+1)
            group_results.append(len(result))

            for key in result:
                if lsh.__contains__(key):
                    print("Similar tweet with id: " + str(key))
                    lsh.remove(key)
            counter += 1

    return groups, group_results

#plots groupings
def plot(group, group_lists):
    fig = plt.figure()
    axes = plt.subplot(1,1,1)
    barWidth=0.50

   
    plt.bar(group, group_lists)
        
        
    axes.set_title('LSH Total Tweets per Group')
    axes.set_ylabel('Group Number')
    axes.set_xlabel('Size')
    plt.show()
    
# # # Main    

if __name__ == '__main__':
    
    # performing LSH
    group, group_lists = get_lsh(128)
    
    #plotting groupings
    plot(group, group_lists)
    
    